# Campus Building Classifier - Demo Inference

This notebook demonstrates how to:
1. Call the backend API `/predict` endpoint
2. Process and visualize prediction results
3. Analyze top-5 predictions

**Prerequisites:**
- Backend running on `http://localhost:8000`
- Sample building image (JPG, PNG, GIF, BMP, or WebP)

## Setup & Imports

In [ ]:
import requests
import json
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Configuration
API_BASE = "http://localhost:8000"
print(f"API Base URL: {API_BASE}")

## Step 1: Check API Health

In [ ]:
# Check if API is running
try:
    response = requests.get(f"{API_BASE}/ping", timeout=5)
    response.raise_for_status()
    print("✓ Backend is healthy")
    print(f"Response: {json.dumps(response.json(), indent=2)}")
except Exception as e:
    print(f"✗ Backend is not reachable: {e}")
    print(f"Make sure the backend is running: uvicorn app.main:app --reload --port 8000")

## Step 2: Get Available Labels

In [ ]:
# Fetch available building labels
try:
    response = requests.get(f"{API_BASE}/labels", timeout=5)
    response.raise_for_status()
    data = response.json()
    labels = data["labels"]
    print(f"✓ Loaded {data['count']} labels:")
    for i, label in enumerate(labels, 1):
        print(f"  {i:2d}. {label}")
except Exception as e:
    print(f"✗ Failed to fetch labels: {e}")

## Step 3: Prepare Test Image

In [ ]:
# TODO: Replace with your actual image path
image_path = "sample_building.jpg"  # Change this to your image path

# For demo, we'll create a simple test image if one doesn't exist
if not Path(image_path).exists():
    print(f"⚠️  Image not found: {image_path}")
    print(f"To test, provide an image and update image_path variable")
    print(f"Supported formats: JPG, PNG, GIF, BMP, WebP")
    
    # Create a dummy image for demonstration
    print(f"\nCreating dummy image for demonstration...")
    dummy_img = Image.new('RGB', (224, 224), color=(100, 150, 200))
    dummy_img.save(image_path)
    print(f"✓ Created dummy image: {image_path}")
else:
    print(f"✓ Image found: {image_path}")

# Display image info
img = Image.open(image_path)
print(f"\nImage Details:")
print(f"  Size: {img.size}")
print(f"  Mode: {img.mode}")
print(f"  Format: {img.format}")

## Step 4: Make Prediction

In [ ]:
# Send image to API and get prediction
try:
    with open(image_path, "rb") as f:
        files = {"file": f}
        response = requests.post(f"{API_BASE}/predict", files=files, timeout=30)
    
    response.raise_for_status()
    prediction = response.json()
    
    print("✓ Prediction successful!")
    print(f"\nPrediction Response:")
    print(json.dumps(prediction, indent=2))
    
except Exception as e:
    print(f"✗ Prediction failed: {e}")
    if hasattr(e.response, 'json'):
        print(f"Error details: {e.response.json()}")

## Step 5: Visualize Results

In [ ]:
# Create visualization of prediction results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Original image
img = Image.open(image_path)
axes[0].imshow(img)
axes[0].set_title(f"Input Image\n({img.size[0]}×{img.size[1]})")
axes[0].axis('off')

# Right: Top-5 predictions bar chart
probs = prediction["probs"]
class_names = [p["class_name"] if "class_name" in p else p["class"] for p in probs]
confidences = [p["confidence"] for p in probs]

colors = plt.cm.Blues(np.linspace(0.4, 0.9, len(class_names)))
bars = axes[1].barh(range(len(class_names)), confidences, color=colors)
axes[1].set_yticks(range(len(class_names)))
axes[1].set_yticklabels(class_names)
axes[1].set_xlabel("Confidence")
axes[1].set_title("Top-5 Predictions")
axes[1].set_xlim(0, 1)

# Add value labels on bars
for i, (bar, conf) in enumerate(zip(bars, confidences)):
    axes[1].text(conf + 0.02, i, f"{conf:.4f}", va='center')

plt.tight_layout()
plt.savefig("prediction_results.png", dpi=100, bbox_inches='tight')
plt.show()

print(f"\n✓ Visualization saved to: prediction_results.png")

## Step 6: Summary & Analysis

In [ ]:
# Print summary
print("="*60)
print("PREDICTION SUMMARY")
print("="*60)
print(f"\nTop Prediction: {prediction['pred']}")
print(f"Confidence: {prediction['confidence']:.4f} ({prediction['confidence']*100:.2f}%)")
print(f"Model Info: {prediction['notes']}")
print(f"\nTop-5 Predictions:")
for i, p in enumerate(prediction['probs'], 1):
    class_name = p["class_name"] if "class_name" in p else p["class"]
    print(f"  {i}. {class_name:30s} {p['confidence']:7.4f} ({p['confidence']*100:6.2f}%)")

if prediction['gradcam_base64']:
    print(f"\n✓ Grad-CAM visualization is available")
else:
    print(f"\n⚠️  Grad-CAM not available (will be added when model is loaded)")

print("="*60)

## Next Steps

1. **Test with multiple images** to see model predictions
2. **Check backend logs** for inference details
3. **Load your trained model** in `backend/app/models/` for real predictions
4. **Implement Grad-CAM** visualization in `inference.py`
5. **Extend metrics** collection for model evaluation

See `backend/app/models/README.txt` for model integration instructions.